# 7506 - Organizacion de Datos - TP N°2

# 1. Librerias

In [1]:
#librerías, no es necesario volverlas a importar

import os
import pandas as pd

import nltk, re, string, collections
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

%run ../0_Data/0_DataLoader.ipynb

[nltk_data] Downloading package punkt to /home/celeste/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/celeste/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/celeste/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# 2. Datos de entrenamiento

In [2]:
x_train_original, y_train = get_train_x_y_original ()

x_train_preprocesado = preprocesar_data (x_train_original, clean_text_sentiment_analysis)


# 3. Datos de test

In [3]:
x_test_original = get_test_x_original()

x_test_preprocesado = preprocesar_data (x_train_original, clean_text_sentiment_analysis)

# 4. Vectorizacion

## 4.1  Obtencion de NGramas, análisis de distintas herramientas

In [4]:
def get_ngrams(documents, size):
    word_vectorizer = CountVectorizer()
    sparse_matrix = word_vectorizer.fit_transform(documents)
    frequencies = sum(sparse_matrix).data
    #frequencies = sum(sparse_matrix).toarray()[0]
    df = pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['frequency'])
    df = df.sort_values(by='frequency', ascending=False)
    return df

In [5]:
%time get_ngrams(x_test_preprocesado,1)

CPU times: user 1.88 s, sys: 4.02 ms, total: 1.88 s
Wall time: 1.88 s


,frequency
know,3276
knight,1979
kno,1947
knowlddg,1830
ks111,1434
...,...
epoch,1
epl,1
epicentr,1
epicent,1


In [6]:
def documentNgrams(documents, size, count=None):
    ngrams_all = []
    for document in documents:
        tokens = document.split()
        if len(tokens) < size:
            continue
        else:
            output = list(ngrams(tokens, size))
        for ngram in output:
            ngrams_all.append(" ".join(ngram))
    cnt_ngram = Counter()
    for word in ngrams_all:
        cnt_ngram[word] += 1
    df = pd.DataFrame.from_dict(cnt_ngram, orient='index').reset_index()
    df = df.rename(columns={'index':'words', 0:'count'})
    df = df.sort_values(by='count', ascending=False)
    if (count == None):
        return df
    else:
        return(df.head(count))

In [7]:
#elijo este que da mejor tiempo de ejecucion y ademas el otro saltea un par de casos. 
# mas adelante se analiza que los casos que saltea son los de letras solas, por ejemplo "u are" 

%time documentNgrams(x_train_preprocesado,1, 100)

CPU times: user 99.3 ms, sys: 0 ns, total: 99.3 ms
Wall time: 98.7 ms


,words,count
3,the,3272
52,a,2177
24,in,1977
22,to,1946
5,of,1825
...,...,...
32,evacu,130
83,see,129
475,some,129
301,train,129


## 4.2  Obtencion de los 100 BiGramas mas frecuentes para todos los tweets

In [8]:
%time bigrams_all_tweets = documentNgrams(x_train_preprocesado,2, 100)

CPU times: user 138 ms, sys: 0 ns, total: 138 ms
Wall time: 137 ms


In [9]:
bigrams_all_tweets.head(10)

,words,count
99,in the,307
90,of the,255
471,like a,130
292,on the,129
1594,to the,125
589,in a,113
489,to be,108
389,for the,97
237,at the,87
1002,and the,79


In [10]:
bigrams_list = bigrams_all_tweets.words.to_list()

In [11]:
word_vectorizer_100 = CountVectorizer(ngram_range=(2, 2), vocabulary = bigrams_list)
sparse_matrix = word_vectorizer_100.fit_transform(x_train_preprocesado.head(10))

In [12]:
print (sparse_matrix.toarray()[1][31])

0


In [13]:
bigrams_list[31]

'this is'

In [14]:
x_train_preprocesado.iloc[1]

'forest fire near la rong sask canada'

## 4.3  Obtencion de los 1000 BiGramas mas frecuentes para todos los tweets

In [15]:
%time bigrams_all_tweets_1000 = documentNgrams(x_train_preprocesado,2, 1000)

CPU times: user 137 ms, sys: 8.02 ms, total: 145 ms
Wall time: 145 ms


In [16]:
bigrams_list_1000 = bigrams_all_tweets_1000.words.to_list()

In [17]:
word_vectorizer_1000 = CountVectorizer(ngram_range=(2, 2), vocabulary = bigrams_list_1000)

In [18]:
vectorizer = CountVectorizer(ngram_range = (2,2), max_features = 5) 
vectorizer.fit_transform(x_train_preprocesado)

<7613x5 sparse matrix of type '<class 'numpy.int64'>'
	with 899 stored elements in Compressed Sparse Row format>

In [19]:
vectorizer.get_feature_names()

['in the', 'of the', 'on the', 'to be', 'to the']

## 4.4  Obtencion de los BiGramas mas frecuentes para los tweets que son desastres 

In [20]:
true_tweets = pd.DataFrame(x_train_preprocesado)
true_tweets = true_tweets.loc[y_train == 1,'text']
true_tweets

id
1        our deed are the reason of this earthquak may ...
4                     forest fire near la rong sask canada
5        all resid ask to shelter in place are be notif...
6        13000 peopl receiv wildfir evacu order in cali...
7        just got sent this photo from rubi alaska as s...
                               ...                        
10869    two giant crane hold a bridg collaps into near...
10870    ahrari the out of control wild fire in califor...
10871                 m194 0104 utc5km s of volcano hawaii
10872    polic investig after an ebik collid with a car...
10873    the latest more home raze by northern californ...
Name: text, Length: 3271, dtype: object

In [21]:
%time bigrams_true_tweets = documentNgrams(true_tweets,2)

CPU times: user 77.4 ms, sys: 27 µs, total: 77.4 ms
Wall time: 76.6 ms


In [22]:
bigrams_true_tweets

,words,count
99,in the,144
90,of the,118
21622,suicid bomber,60
1110,on the,51
370,in a,50
...,...,...
10039,neat websit,1
10038,pretti neat,1
10037,80m peopl,1
10036,affect 80m,1


## 4.5  Obtencion de los TriGramas mas frecuentes para los tweets que son desastres 

In [23]:
%time trigrams_true_tweets = documentNgrams(true_tweets,3)

CPU times: user 68.1 ms, sys: 5 µs, total: 68.2 ms
Wall time: 67.6 ms


In [24]:
trigrams_true_tweets

,words,count
22023,home raze by,29
22022,more home raze,29
16578,northern california wildfir,29
22020,the latest more,28
24296,deton bomb in,28
...,...,...
10507,peopl were displacedinjuredkil,1
10506,of peopl were,1
10505,thousand of peopl,1
10504,amtalia thousand of,1


## 4.6  Obtencion del vocabulario de los tweets que son desastres naturales  

In [25]:
%time vocabulary_true_tweets = documentNgrams(true_tweets,1)

CPU times: user 54.7 ms, sys: 48 µs, total: 54.8 ms
Wall time: 54.7 ms


In [26]:
vocabulary_true_tweets

,words,count
3,the,1361
24,in,1159
52,a,925
5,of,923
22,to,757
...,...,...
3805,furi,1
3803,load,1
3802,paratroop,1
3800,humphrey,1


In [27]:
true_tweets = pd.DataFrame(x_train_original)
true_tweets = true_tweets.loc[y_train == 1,'text']
true_tweets

id
1        Our Deeds are the Reason of this #earthquake M...
4                   Forest fire near La Ronge Sask. Canada
5        All residents asked to 'shelter in place' are ...
6        13,000 people receive #wildfires evacuation or...
7        Just got sent this photo from Ruby #Alaska as ...
                               ...                        
10869    Two giant cranes holding a bridge collapse int...
10870    @aria_ahrary @TheTawniest The out of control w...
10871    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
10872    Police investigating after an e-bike collided ...
10873    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 3271, dtype: object

In [28]:
vocabulary_true_tweets_original = documentNgrams(df_train.loc[(df_train['target']== 1),'text'],1)

NameError: name 'df_train' is not defined

In [ ]:
vocabulary_true_tweets_original